**STEPS**
1. Data select via SQL
2. Data cleaning
3. Converting categorical vector into binary via OneHotEncoding (OHE)
4. Drop out useless columns after OHE: housing_no', 'loan_no', 'client_id', 'education_unknown', 'poutcome_other','poutcome_failure', 'y_no', 'job_unknown'
5. Data standardization and normalization via SMOTE
6. Train predictive models: Logistic Regression, SVM, Neuron Networks, Random Forest
7. Create 2 data client group with different logic and compare its ratio between probability and estimated conversion value
8. Calculate of estimated conversion rate and value for final selected top 5000 clients 

**OVERVIEW**

Our commitment to data-driven marketing has yielded valuable insights to optimize our campaign. We employed advanced techniques like one-hot encoding, SMOTE, and A/B testing to refine our targeting strategy and best data model.

This analysis revealed a critical finding: prioritizing candidates with a high probability of conversion (Group 2) holds significant promise. Compared to the initial focus on high expected revenue alone (Group 1), Group 2 demonstrates best ratio between probability and estimated conversion value.

These data-driven results make us confident that the campaign will achieve successful outcomes.





In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('/content/data.db')

# Data retrieve from given dataset into DataFrame
dt_select = '''
SELECT DISTINCT * from Accounts a
LEFT JOIN Campaigns c on a.id = c.account_id
LEFT JOIN Clients cl on a.client_id = cl.id
LEFT JOIN Outcomes o on o.campaign_id = c.id
WHERE a.in_default = 'no' AND c.pdays != -1 AND poutcome != 'unknown'
'''

df = pd.read_sql_query(dt_select, conn)

# Close connection
conn.close()

df.head(10)

In [ ]:
# We dropped the columns that we do not find as important
df.drop(columns=(['account_id', 'campaign_id','in_default', 'contact', 'id', 'day', 'month', 'duration', 'previous', 'campaign', 'pdays']), inplace=True, axis=0)


In [ ]:
# Saving df into csv file
df.to_csv('1_phase_5.csv', index=False)

Converting categorical vector into binary via OneHotEncoding (OHE)

In [ ]:
# One hot encoding
columns_to_encode = ['housing', 'loan', 'job', 'marital', 'education', 'poutcome', 'y']

df_encoded = pd.get_dummies(df, columns=columns_to_encode)
#df_encoded.tail()

boolean_columns = df_encoded.select_dtypes(include=['bool']).columns
df_encoded[boolean_columns] = df_encoded[boolean_columns].astype(int)

df_encoded.head()
df_encoded.info()

In [ ]:
df_encoded.head()
# saving the OHE data
df_encoded.to_csv('1_phase_df_encoded.csv', index=False)

In [ ]:
import pandas as pd

# Read the CSV file into a dataframe
df = pd.read_csv('1_phase_df_encoded.csv')

# Display the dataframe
df.head()

In [ ]:
df = df[~(df['poutcome_other'] == 1)]


Drop out useless columns after OHE: housing_no', 'loan_no', 'client_id', 'education_unknown', 'poutcome_other','poutcome_failure', 'y_no', 'job_unknown'

In [ ]:
columns_to_drop = ['housing_no', 'loan_no', 'client_id', 'education_unknown', 'poutcome_other','poutcome_failure', 'y_no', 'job_unknown']
df = df.drop(columns=columns_to_drop)


Data standardization and normalization via SMOTE method

In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd

# Rename dataframe to match user's dataframe name

# Separate features and target variable
X = df.drop(columns=['y_yes'])
y = df['y_yes']

# Apply SMOTE for over-sampling
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Combine resampled features and target variable
df_resampled = pd.concat([X_res, y_res], axis=1)

# Check the distribution of the target variable to ensure balance
resampled_distribution = df_resampled['y_yes'].value_counts()
print(resampled_distribution)

In [ ]:
df.head()

In [ ]:
df_resampled.to_csv('df_SMOTE.csv', index=False)

TRAIN MODELS:
1. Logistic Regression
2. SVM
3. Neuron Networks
4. Random Forest

1. Logistic Regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Load Data and Define Target Variable
data = pd.read_csv('df_SMOTE2.csv')

target = 'y_yes'  # Assuming 'y_yes' is the target variable

# Step 2: Preprocess Data

# Check for missing values
missing_values = data.isnull().sum().sum()
print("Missing values:", missing_values)

# Handle missing values if there are any (e.g., fillna or dropna)
# data = data.fillna(method='ffill')  # Example of filling missing values
# Separate features and target
X = data.drop(target, axis=1)
y = data[target]

# Ensure target variable is suitable for classification (optional)
print("Target variable data type:", y.dtype)

# Standardize data (recommended for weighted classifiers)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.6, random_state=42)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Step 3: Train and Evaluate Logistic Regression Model

# Define and train the Logistic Regression model
model = LogisticRegression(class_weight=class_weights_dict)
model.fit(X_train, y_train)

# Make predictions on test set
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Logistic Regression - Accuracy:", accuracy)
print("Logistic Regression - Precision:", precision)
print("Logistic Regression - Recall:", recall)
print("Logistic Regression - F1-score:", f1)

##############################################################

# Step 4: Identify Top 5000 Likely Buyers

# Predict probabilities for the entire dataset using Logistic Regression
probabilities = model.predict_proba(X_scaled)[:, 1]

# Create a DataFrame with the original data and the predicted probabilities
data_with_prob = data.copy()
data_with_prob['probability'] = probabilities

# Sort the DataFrame by probability in descending order
data_with_prob_sorted = data_with_prob.sort_values(by='probability', ascending=False)

# Select the top 5000 individuals
top_5000 = data_with_prob_sorted.head(5000)

# Display the top 5000 individuals
print(top_5000)

# Optionally, save the top 5000 individuals to a CSV file
top_5000.to_csv('top_5000_customers.csv', index=False)



2. SVM with using GridSearchCV with 5- fold and 10-fold cross-validation in pipelines

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve

# Load data
data = pd.read_csv('/content/df_SMOTE2.csv')

#  Separate features and target
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline for model training
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True))  # Setting probability=True
])

# Define parameter grid
param_grid = {
    'svm__kernel': ['linear', 'rbf', 'poly'],
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto']
}

# GridSearchCV with 10-fold cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Prediction and evaluation of model
y_pred = grid_search.best_estimator_.predict(X_test)

print("Best parameters found:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Compute and display the confusion matrix
con_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(con_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Probabality prediction for whole dataset
scaler = grid_search.best_estimator_.named_steps['scaler']
svm_model = grid_search.best_estimator_.named_steps['svm']
y_prob_full = svm_model.predict_proba(scaler.transform(X))

print("Predicted probabilities for the entire dataset:\n", y_prob_full)

# Evaluation- ROC AUC score
y_prob = svm_model.predict_proba(scaler.transform(X_test))
roc_auc = roc_auc_score(y_test, y_prob[:, 1])
print("ROC AUC Score:", roc_auc)

# Evaluation- ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob[:, 1])
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


# Prediction of new data
new_data = pd.read_csv('/content/to_clean_cleaned.csv')
new_data_drop = new_data.drop(columns=['client_id', 'y_yes'])


# Probabality prediction for new data
new_predictions = grid_search.best_estimator_.predict_proba(new_data_drop)
print("New Predictions Probabilities:\n", new_predictions)



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Loaded dataset
data = pd.read_csv('/content/df_SMOTE2.csv')

# Data are devided based on the last column: 'y_yes'
X = data.iloc[:, :-1]  # all columns except 'y_yes' column
y = data.iloc[:, -1]   # 'y_yes' column

# Setting data into the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.model_selection import GridSearchCV

# Define parameter grid for GrindSearchCV
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

# GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(SVC(probability=True), param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best parameters found:", best_params)

# Prediction and evaluation of model
y_pred_best = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Classification Report:\n", classification_report(y_test, y_pred_best))


3. Neuron Networks

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Loaded dataset
data = pd.read_csv('/content/df_SMOTE2.csv')

# Data are devided based on the last column: 'y_yes'
X = data.iloc[:, :-1]  # all columns except 'y_yes' column
y = data.iloc[:, -1]   # 'y_yes' column

# Setting data into the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=42)

perceptron = Perceptron(max_iter=5, random_state=42)

perceptron.fit(X_train, y_train)

y_pred = perceptron.predict(X_test)
print(y_pred)

print(classification_report(y_test, y_pred))

4. Random Forest with using GridSearchCV with 5-fold cross-validation and KFold with 5-fold cross-validation in pipelines

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

# Load the dataset
data = pd.read_csv('df_SMOTE2.csv')
X = data.iloc[:, :-1] # Features
y= data.iloc[:, -1] # Target variable

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest classifier with 25 trees
clf_25 = RandomForestClassifier(n_estimators=25, criterion='log_loss', min_samples_leaf=4, random_state=42)
clf_25.fit(X_train, y_train)
y_pred_25 = clf_25.predict(X_test)
accuracy_25 = accuracy_score(y_test, y_pred_25)
print("Classification Report:\n", classification_report(y, y_pred_25))
print("Accuracy with 25 trees: {:.2f}%".format(accuracy_25 * 100))

# Random Forest classifier with 50 trees
clf_50 = RandomForestClassifier(n_estimators=50, criterion='log_loss', min_samples_leaf=4, random_state=42)
clf_50.fit(X_train, y_train)
y_pred_50 = clf_50.predict(X_test)
accuracy_50 = accuracy_score(y_test, y_pred_50)
print("Classification Report:\n", classification_report(y, y_pred_50))
print("Accuracy with 50 trees: {:.2f}%".format(accuracy_50 * 100))

# Random Forest classifier with 75 trees
clf_75 = RandomForestClassifier(n_estimators=75, criterion='log_loss', min_samples_leaf=4, random_state=42)
clf_75.fit(X_train, y_train)
y_pred_75 = clf_75.predict(X_test)
accuracy_75 = accuracy_score(y_test, y_pred_75)
print("Classification Report:\n", classification_report(y, y_pred_75))
print("Accuracy with 75 trees: {:.2f}%".format(accuracy_75 * 100))

# Create a Random Forest classifier with 200 trees
clf_200 = RandomForestClassifier(n_estimators=200, criterion='log_loss', min_samples_leaf=4, random_state=42)
clf_200.fit(X_train, y_train)
y_pred_200 = clf_200.predict(X_test)
accuracy_200 = accuracy_score(y_test, y_pred_200)
print("Classification Report:\n", classification_report(y, y_pred_200))
print("Accuracy with 200 trees: {:.2f}%".format(accuracy_200 * 100))

# Compute and display the confusion matrix of winning Random Forest with 200 trees
cm = confusion_matrix(y, y_pred_200)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap='Blues')

# Show the plot
plt.show()

In [ ]:
# Using Stratified K-Fold cross-validation for estimating predictive performance of Random Forest model with 200 trees (unbalanced dataset)
# Initialize Stratified K-Fold with 5 splits
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and get predictions
y_pred_200 = cross_val_predict(clf_200, X, y, cv=skf)

# Compute and print the accuracy
accuracy_skf = accuracy_score(y, y_pred)
print("Overall Accurancy: {:.2f}%".format(accuracy_skf * 100))

In [ ]:
# GridSearchCV with 5-fold cross-validation
pipe_rf = Pipeline([
    ('pca', PCA(n_components=2)),
    ('clf', GridSearchCV(
        estimator=RandomForestClassifier(random_state=42),
        param_grid={
            "n_estimators": [75, 50, 100,200],
            "min_samples_split": [2, 3, 4, 5],
            "min_samples_leaf": [1,2,3]},
        cv=5, verbose=0)
    )
])
pipe_rf.fit(X_train,y_train)
y_pred = pipe_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy after PCA and GridSearchCV: {:.2f}%".format(accuracy * 100))

In [ ]:
# Save the new dataset with predictions
output_file_path = 'new_data_with_predictions.csv'
new_data.to_csv(output_file_path, index=False)

print(f"Predictions have been added to the new dataset and saved to {output_file_path}.")

Select top 5000 clients with high perfomance of estimated conversion value

Create 2 data client groups and compare its ratio between probability and estimated conversion value

**Group 1** - Sorted by expected_revenue descending with probability >= 0,3

**Group 2** - Sorted by probability only descending, demonstrates best ratio between probability and estimated conversion value => final select by this method







In [ ]:
import pandas as pd
from IPython.display import display
import shutil

# Load dataset
file_path = 'Sorted_Client_Data.csv'
df = pd.read_csv(file_path)

# Drop dupliccates by client_id
df = df.drop_duplicates(subset='client_id')

# Convert expected_revenue into numeric datatype
df['expected_revenue'] = pd.to_numeric(df['expected_revenue'], errors='coerce')

# Filtering clients who are not 'in_default'
df = df[df['in_default'] == 'no']

# Create group1
group1 = df[df['prob_1'] >= 0.3].sort_values(by='expected_revenue', ascending=False)

# Create group2
group2 = df.sort_values(by='prob_1', ascending=False)


# Create ratio for comparison between group1 and group2
total_clients = 5000
results = []

for g1_ratio in range(11):
    g2_ratio = 10 - g1_ratio
    n_g1 = (g1_ratio * total_clients) // 10
    n_g2 = total_clients - n_g1

    selected_group1 = group1.head(n_g1)
    selected_group2 = group2.head(n_g2)

    combined_group = pd.concat([selected_group1, selected_group2]).drop_duplicates(subset='client_id').head(total_clients)

    avg_prob_1 = combined_group['prob_1'].mean()
    total_expected_revenue = combined_group['expected_revenue'].sum()

    results.append({
        'group1_ratio': g1_ratio / 10,
        'group2_ratio': g2_ratio / 10,
        'avg_prob_1': avg_prob_1,
        'total_expected_revenue': total_expected_revenue
    })

# Convert into Dataframe for analysis
results_df = pd.DataFrame(results)
display(results_df)

# Select top 5000 clients by prob_1 descending
top_5000_clients = df.nlargest(5000, 'prob_1')

# Calculate average probability and total expected revenue
average_prob_1 = top_5000_clients['prob_1'].mean()
total_expected_revenue = top_5000_clients['expected_revenue'].sum()

print(f'Average prob_1: {average_prob_1}')
print(f'Total expected revenue: {total_expected_revenue}')

# Save top 5000 clients into csv file and create dowload function
output_file_path = 'Top_5000_Clients.csv'
top_5000_clients.to_csv(output_file_path, index=False)

def download_file(file_path, output_path):
    shutil.copy(file_path, output_path)
    print(f'Soubor byl uložen na: {output_path}')

Calculate of estimated conversion rate and value for final select top 5000 clients


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Load the CSV file
data = pd.read_csv('Top_5000_Clients.csv')

# Display the first few rows of the dataframe
print(data.head())

def calculate_estimated_conversion(df):

    # Calculate the estimated conversion rate
    estimated_conversion_rate = df['prob_1'].mean()

    # Calculate the estimated conversion value
    estimated_conversion_value = (df['prob_1'] * df['monthly_income']).sum()

    return estimated_conversion_rate, estimated_conversion_value

# Perform the calculation
estimated_conversion_rate, estimated_conversion_value = calculate_estimated_conversion(data)

print(f"Estimated Conversion Rate: {estimated_conversion_rate * 100:.2f}%")
print(f"Estimated Conversion Value: {estimated_conversion_value:.2f}")